In [ ]:
import transformers
import pandas as pd
import datasets
import faiss
import numpy as np

/home/oumar/Projects/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datasets import Dataset

dataset = pd.read_csv("data/data1.csv")
dataset.drop(columns=["Unnamed: 0"], inplace=True)

data = Dataset.from_pandas(dataset)
data

Dataset({
    features: ['question', 'answer'],
    num_rows: 11697
})

In [ ]:
from transformers import  SentenceTransformer

encoder = SentenceTransformer('avsolatorio/GIST-small-Embedding-v0')

In [5]:
qa_pairs = [{"question": q, "answer": a} for q, a in zip(data["question"], data["answer"])]
qa_pairs

[{'question': 'What is Natural Language Processing (NLP)?',
  'answer': 'Natural Language Processing is a field of artificial intelligence that focuses on the interaction between computers and human languages, enabling machines to understand, interpret, and generate human-like text.'},
 {'question': 'How does NLP relate to Computer Vision?',
  'answer': 'NLP and Computer Vision often complement each other in tasks where understanding both textual and visual information is crucial. For example, in image captioning, the combination of NLP and Computer Vision is used to generate descriptive captions for images.'},
 {'question': 'What is the role of tokenization in NLP?',
  'answer': 'Tokenization is the process of breaking down text into smaller units called tokens. In NLP, tokens are typically words or subwords. Tokenization is a fundamental step in various NLP tasks, helping to analyze and understand the structure of the text.'},
 {'question': 'Can you explain the concept of word embedd

In [ ]:
question_text = [qa["question"] for qa in qa_pairs]
answer_text = [qa["answer"] for qa in qa_pairs]

question_embeddings = encoder.encode(question_text)
answer_embeddings = encoder.encode(answer_text)

# Convert to faiss
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))

In [ ]:
# Retrive similar questions
def retriev_similar_questions(query, top_k=5):
    query_embedding = encoder.encode(query)
    _, similar_question_idxs = index.search(np.array(query_embedding), top_k)
    similar_questions = [question_text[i] for i in similar_question_idxs[0]]
    return similar_questions